In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
serper_api_key = os.getenv('SERPER_API_KEY')

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an instance of the OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(api_key=api_key, model="gpt-4o")

In [3]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool


search = GoogleSerperAPIWrapper()

serper_tool = Tool(
    name="Search",
    func=search.run,
    description="useful for when you need to ask with search",
)
    

In [22]:
from autogen_ext.tools.langchain import LangChainToolAdapter

web_search_tool = LangChainToolAdapter(serper_tool)

storyteller_agent = AssistantAgent(
    name="STORYTELLER_AGENT",
    model_client=model_client,
    system_message="You are a helpful storyteller. Write stories in less than 100 words.",
)

quote_agent = AssistantAgent(
    name="QUOTE_AGENT",
    model_client=model_client,
    tools=[web_search_tool],
    system_message='''
    You are a web search agent that MUST always use the web_search_tool.
    
    CRITICAL INSTRUCTIONS:
    - You MUST use the web_search_tool for every request
    - NEVER respond without first making a web search
    - Your workflow is: 1) Use web_search_tool, 2) Then provide response based on results
    - If asked for a quote, search for "[topic] quotes" or similar
    - Make exactly one search call per request
    - Always wait for search results before responding
    
    Do not provide any response until you have used the web_search_tool first.
    '''
)

story_improvisor_agent = AssistantAgent(
    name="STORY_IMPROVISOR_AGENT",
    model_client=model_client,
    system_message="""
    You are a helpful story improviser. 
    Your job is to take the story and quotes and improve the story by adding the quotes in less than 100 words.
    After all the tasks are completed end with 'TERMINATE'
    """,
)                                                                                                           


In [23]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination


# Termination Condition
termination_condiation = TextMentionTermination('TERMINATE')


# Create the group chat.
team = RoundRobinGroupChat(
    participants=[storyteller_agent, quote_agent, story_improvisor_agent],
    termination_condition=termination_condiation
)

In [25]:
from autogen_agentchat.ui import Console

await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a story about Superman."))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a story about Superman.
---------- TextMessage (STORYTELLER_AGENT) ----------
Superman soared over Metropolis, his cape a red trail against the evening sky. Below, chaos reigned as Lex Luthor had unleashed robotic minions to sow fear. With precision, Superman descended, dismantling each robot with swift blows. As Lex watched from his tower, a smirk faded to frustration; his plan was thwarted yet again.

"Why do you keep fighting, Superman?" Lex taunted via hologram.

"Because hope never ends," Superman replied, eyes blazing with determination.

With the city safe, Superman paused, gazing at the grateful faces below. For him, being a hero was instinctive—a reminder that even with power, kindness prevailed.
---------- ToolCallRequestEvent (QUOTE_AGENT) ----------
[FunctionCall(id='call_80iqR2oLsc76hLgSNRadUrgD', arguments='{"query":"Superman story"}', name='Search')]
---------- ToolCallExecutionEvent (QUOTE_AGENT) ----------
[FunctionExecuti

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a story about Superman.', type='TextMessage'), TextMessage(source='STORYTELLER_AGENT', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=136), metadata={}, content='Superman soared over Metropolis, his cape a red trail against the evening sky. Below, chaos reigned as Lex Luthor had unleashed robotic minions to sow fear. With precision, Superman descended, dismantling each robot with swift blows. As Lex watched from his tower, a smirk faded to frustration; his plan was thwarted yet again.\n\n"Why do you keep fighting, Superman?" Lex taunted via hologram.\n\n"Because hope never ends," Superman replied, eyes blazing with determination.\n\nWith the city safe, Superman paused, gazing at the grateful faces below. For him, being a hero was instinctive—a reminder that even with power, kindness prevailed.', type='TextMessage'), ToolCallRequestEvent(source='QUOTE_AGENT', models_usage=Reques